In [11]:
from bs4 import BeautifulSoup
import requests

In [ ]:
url = "http://www.xssed.com/archive/page={}/"

In [4]:
mirrors = []
for i in range(1,1530):
    r = requests.get(url.format(i))
    soup = BeautifulSoup(r.content,"html5lib")

    for link in soup.find_all('a'):
        if "mirror" in link.get("href"):
            mirrors.append(link.get("href"))

In [ ]:
url = "http://www.xssed.com{}"
malicious_urls = []
mirror_urls = []

for m in mirrors:
    mirror = url.format(m)
    r = requests.get(mirror)
    soup = BeautifulSoup(r.content,"html5lib")
    malicious_urls.append(soup.select("#contentpaneOpen > table > tbody > tr:nth-child(4) > th")[0].text[5:])
    mirror_urls.append(soup.select("#contentpaneOpen > table > tbody > tr:nth-child(5) > th > a")[0].get("href"))

In [8]:
def write_file(filename,strings):
    with open(filename, "w") as file:
        for string in strings:
            file.write(string + "\n")

write_file("mirrors.txt",mirror_urls)
write_file("malicious_urls.txt",malicious_urls)

In [5]:
def file_to_list(filename):
    lines = []
    with open(filename,'r') as file:
        for line in file:
            cleaned_line = line.strip()
            lines.append(cleaned_line)
    return lines

In [15]:
import csv
import random

data = []
def generate_csv():
    malicious = file_to_list("malicious_urls.txt")
    benign = file_to_list("benign_urls.txt")
    for m in malicious:
        data.append([m,1])
    for b in benign:
        data.append([b,0])
    random.shuffle(data)

    with open("urls.csv",mode="w",newline="") as file:
        csv_writer = csv.writer(file)
        for row in data:
            print(row)
            csv_writer.writerow(row)
    file.close()

In [ ]:
generate_csv()

In [19]:
def read_csv(filename):
    with open(filename,mode="r") as file:
        csv_reader = csv.reader(file)
        data = []
        for row in csv_reader:
            data.append(row)
        return data

In [38]:
import re

def duplicate(url):
    patterns = [r'<<', r'>>', r'%3C%3']
    total_occurrences = 0
    for pattern in patterns:
        occurrences = len(re.findall(pattern, url))
        total_occurrences += occurrences
    return total_occurrences

def special(url):
    substrings = ["“, “”>", "“>", "“/>"]
    total_occurrences = 0
    for substring in substrings:
        total_occurrences += url.count(substring)
    return total_occurrences

def scripts(url):
    pattern = re.compile("script", re.IGNORECASE)
    matches = pattern.findall(url)
    return len(matches)

def cookie(url):
    pattern = r'document\.cookie'
    if re.search(pattern, url):
        return 1
    return 0

def redirections(url):
    pattern = r'window\.(location\.href|history\.back|navigate)'
    if re.search(pattern, url):
       return 1
    return 0

def extract_features(url):
    return [len(url),duplicate(url),special(url),scripts(url),cookie(url),redirections(url)]

In [40]:
dataset = [["number_of_characters","duplicate_characters","scripts","cookie_requests","redirections","target"]]

with open("urls.csv",mode="r") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        new_row = extract_features(row[0])
        new_row.append(int(row[1]))
        dataset.append(new_row)
file.close()

with open("urls_features_data.csv",mode="w",newline="") as file:
    csv_writer = csv.writer(file)
    for row in dataset:
        csv_writer.writerow(row)

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv("urls_features_data.csv")
X = df.drop('target',axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [48]:
y_pred = clf.predict(X_test)


In [49]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9738146450949219
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7069
           1       0.98      0.94      0.96      3624

    accuracy                           0.97     10693
   macro avg       0.98      0.96      0.97     10693
weighted avg       0.97      0.97      0.97     10693



In [50]:
from sklearn.svm import SVC

clf = SVC(kernel='linear', C=1)  # Use a linear kernel (you can choose other kernels)
clf.fit(X_train, y_train)

SVC(C=1, kernel='linear')

In [52]:
y_pred = clf.predict(X_test)


In [54]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9738146450949219
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7069
           1       0.98      0.94      0.96      3624

    accuracy                           0.97     10693
   macro avg       0.98      0.96      0.97     10693
weighted avg       0.97      0.97      0.97     10693



In [55]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=42)  # You can specify other hyperparameters
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.9738146450949219
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7069
           1       0.98      0.94      0.96      3624

    accuracy                           0.97     10693
   macro avg       0.98      0.96      0.97     10693
weighted avg       0.97      0.97      0.97     10693

